In [1]:
import tensorflow as tf
import pandas as pd
from datetime import time
import numpy as np
from sklearn.model_selection import train_test_split


/home/admin-/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
df = pd.read_pickle('./df_dataset.pkl')
df = df.sample(frac=1)
df_train = df.head(int(len(df)/0.8))
df_test = df.tail(int(len(df)/0.2))

In [3]:
df_train = df.head(int(len(df)/0.8))
df_test = df.tail(int(len(df)/0.2))

In [4]:
x_train = df_train.seq.values
y_train = df_train.y.values
x_test = df_test.seq.values
y_test = df_test.y.values

In [5]:
config = {
    'lr':0.001,
    'n_hidden':128,
    'total_epoch':10,
    'batch_size':256,
    'embedding_size':128,
    'n_eval':10
}
lr = config['lr']
n_hidden = config['n_hidden']
total_epoch = config['total_epoch']
batch_size = config['batch_size']
embedding_size = config['embedding_size']
# self.vocabulary_list = data_helper.vocabulary_list
# self.vocabulary_dict = data_helper.vocabulary_dict
n_class = 2
n_eval = config['n_eval']
training_mode = True
n_vocab = len(pd.read_pickle('./df_vocabulary.pkl'))

global_step = tf.Variable(0, name = 'global_step', trainable = False)
output_keep_prob = tf.placeholder(tf.float32)
current_batch_size = tf.placeholder(dtype = tf.int32, shape = [], name = "current_batch_size")

encoder_inputs = tf.placeholder(dtype = tf.int32, shape = [None, None], name = "encoder_inputs")
decoder_inputs = tf.placeholder(dtype = tf.int64, shape = [None, None], name = "decoder_inputs")
decoder_outputs = tf.placeholder(tf.int64, [None], name = "decoder_outputs")


In [6]:
with tf.variable_scope('embedding'):
    embedding = tf.get_variable("embedding_layer", [n_vocab, embedding_size], trainable = True)
    encoder_emb_inp = tf.nn.embedding_lookup(embedding, encoder_inputs)
    # decoder_emb_inp = tf.nn.embedding_lookup(embedding, decoder_inputs)
    
with tf.variable_scope('encode'):
    encoder_length = tf.placeholder(tf.int32, [None], name = "encoder_length")    
    enc_cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
    enc_cell = tf.nn.rnn_cell.DropoutWrapper(enc_cell, output_keep_prob = output_keep_prob)
    outputs, enc_states = tf.nn.dynamic_rnn(cell = enc_cell, inputs = encoder_emb_inp,
                                                      dtype = tf.float32, sequence_length = encoder_length)
    
with tf.variable_scope('attention'):
    attention_mechanism = tf.contrib.seq2seq.LuongAttention(
        n_hidden, outputs, memory_sequence_length = encoder_length)

with tf.variable_scope('decode'):
    decoder_length = tf.placeholder(tf.int32, [None], name = "decoder_length")
    target_weights = tf.placeholder(tf.float32, [None, None], name = "target_weights")

    dec_cell = tf.nn.rnn_cell.LSTMCell(n_hidden)
    dec_cell = tf.nn.rnn_cell.DropoutWrapper(dec_cell, output_keep_prob = output_keep_prob)

    dec_cell = tf.contrib.seq2seq.AttentionWrapper(
        dec_cell, attention_mechanism, attention_layer_size = n_hidden)
    helper = tf.contrib.seq2seq.TrainingHelper(decoder_emb_inp, decoder_length)
    projection_layer = tf.layers.Dense(n_vocab, use_bias = True)
    decoder = tf.contrib.seq2seq.BasicDecoder(
        dec_cell, helper,
        initial_state = dec_cell.zero_state(current_batch_size, tf.float32)
            .clone(cell_state = enc_states),
        output_layer = projection_layer)
    outputs, dec_states, final_sequence_lengths = tf.contrib.seq2seq.dynamic_decode(decoder)
            


Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').


In [ ]:
'''you need to build enc, dec or etc in drived classes '''
with tf.variable_scope('output'):
    logits = outputs.rnn_output
    prediction = tf.argmax(logits, axis = 2)

with tf.variable_scope('Cost'):
    crossent = tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits = logits, labels = decoder_outputs)
    cost = (tf.reduce_mean(crossent * target_weights))
    tf.summary.scalar('cost', cost)

with tf.variable_scope('Accuracy'):
    # predictions = prediction * target_weights
    correct_predictions = tf.equal(prediction, decoder_outputs)
    accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"))
    tf.summary.scalar('accuracy', accuracy)

with tf.variable_scope('optimiser'):
    params = tf.trainable_variables()
    gradients = tf.gradients(cost, params)
    clipped_gradients, _ = tf.clip_by_global_norm(gradients, 1)
    lr = tf.train.exponential_decay(lr , global_step = global_step, decay_steps = n_eval,
                                    decay_rate = 0.999, staircase = True)
    opimiser = tf.train.AdamOptimizer(lr)
    train_op = opimiser.apply_gradients(
        zip(clipped_gradients, params), global_step = global_step)

graph = tf.Graph()
saver = tf.train.Saver()
sess = tf.Session()
train_writer = tf.summary.FileWriter('./tensorboard', sess.graph)
sess.run(tf.global_variables_initializer())
set_checkpoints()

In [ ]:
# def load_data_set(self, file_name):
#     if not file_name:
#         file_name = 'with_noise'
#     self.df_train = pd.read_csv('./dataset/df_train_' + file_name + '.csv')
#     self.df_test = pd.read_csv('./dataset/df_test_' + file_name + '.csv')
# 
def set_checkpoints(self, timestamp = None):
    # Checkpoint files will be saved in this directory during training
    if not timestamp:
        timestamp = str(int(time.time()))

    self.timestamp = timestamp
    self.checkpoint_dir = './checkpoints_' + self.timestamp + '/'
    # if os.path.exists(self.checkpoint_dir):
    #     shutil.rmtree(self.checkpoint_dir)
    # os.makedirs(self.checkpoint_dir)
    self.checkpoint_prefix = os.path.join(self.checkpoint_dir, 'model')
    
    
def set_feed_dict(self, batch, is_train = False):
        enc_input_batch, dec_input_batch, dec_output_batch, target_weights_batch, \
        enc_len_batch, dec_len_batch, current_batch_size_batch \
            = data_helper.make_batch(pd.DataFrame(batch, columns = ['x', 'y']))
        if is_train:
            output_keep_prob = 0.75
        else:
            output_keep_prob = 1

        feed_dict = {
            self.encoder_inputs:enc_input_batch,
            self.decoder_inputs:dec_input_batch,
            self.decoder_outputs:dec_output_batch,
            self.target_weights:target_weights_batch,
            self.encoder_length:enc_len_batch,
            self.decoder_length:dec_len_batch,
            self.output_keep_prob:output_keep_prob,
            self.current_batch_size:current_batch_size_batch
        }
        return feed_dict

In [ ]:
def train(self):
    x_train = self.df_train.x.tolist()
    y_train = self.df_train.y.tolist()
    train_batches = data_helper.batch_iter(
        data = list(zip(x_train, y_train)), batch_size = self.batch_size, num_epochs = self.total_epoch)
    train_loss, train_best_loss, val_best_loss, self.best_at_step = 0, 100, 100, 0

    log_msg_list = []

    for train_batch in train_batches:
        current_step = tf.train.global_step(self.sess, self.global_step)
        feed_dict = self.set_feed_dict(train_batch, True)
        self.merged_summaries = tf.summary.merge_all()
        _, loss, accuracy, summary = self.sess.run(
            [self.train_op, self.cost, self.accuracy, self.merged_summaries], feed_dict = feed_dict)
        self.train_writer.add_summary(summary = summary, global_step = current_step)

        log_msg = 'current_step = ', '{}'.format(current_step), \
                  ', cost = ', '{:.6f}'.format(loss), \
                  ', accuracy = ', '{:.6f}'.format(accuracy)+ '\n'
        log_msg_list += log_msg
        print(log_msg)

        train_loss += loss

        if current_step != 0 and current_step % self.n_eval == 0:
            val_loss, val_accuracy = self.test(
                self.df_test.sample(frac = 0.2).reset_index(drop = True), False)
            train_loss /= (self.n_eval)
            log_msg = 'current_step = ', '{}'.format(current_step), \
                      ', val_cost = ', '{:.6f}'.format(val_loss), \
                      ', val_accuracy = ', '{:.6f}'.format(val_accuracy), \
                      ', train_cost = ', '{:.6f}'.format(train_loss)+ '\n'
            log_msg_list += log_msg
            print(log_msg)

            if train_loss < train_best_loss and val_loss < val_best_loss:
                train_best_loss, val_best_loss, self.best_at_step = train_loss, val_loss, current_step

                self.save_current_session(current_step)

                print('Best cost {:.6f} and {:.6f} at step {}'.format(
                    train_best_loss, val_best_loss, self.best_at_step))

            with open('./logs/log_' + self.timestamp + '.txt', 'a') as f:
                f.writelines(log_msg_list)

            log_msg_list = []
            train_loss = 0
            # conf = tf.ConfigProto()

In [ ]:
@abc.abstractmethod
def save_current_session(self, current_step):
    pass

@abc.abstractmethod
def restore_best_session(self, best_at_step = None):
    pass

def test(self, df, file_name, is_eval = True):
    feed_dict = self.set_feed_dict(df, True)

    results, loss, accuracy = self.sess.run([self.prediction, self.cost, self.accuracy], feed_dict = feed_dict)

    if is_eval:
        return loss, accuracy
    else:
        print('cost = ', '{:.6f}'.format(loss), ', accuracy = ', '{:.6f}'.format(accuracy))

        decoded_number = []
        for result in results:
            decoded_number.append([self.vocabulary_list[i] for i in result])

        decoded_jamo = []
        for result in decoded_number:
            try:
                end = result.index('E')
                decoded_jamo.append(''.join(result[:end]))
            except:
                decoded_jamo.append(''.join(result))

        df['predict'] = [jamo.join_jamos(x) for x in decoded_jamo]
        df.to_csv('./model_result/' + file_name + '_result_' + self.timestamp + '.csv', index = False)
tf.keras.layers.Dense()